#### Импорт необходимых библиотек

In [1]:
# Импорт необходимых библиотек для дальнейшей работы
import json
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import warnings

from collections.abc import Mapping, Sequence
from itertools import chain
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from pathlib import Path
from regex import Pattern, compile as re_compile
from scipy import sparse
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from spacy import load as spacy_load_model
from spacy.cli import download as spacy_download_model
from typing import Any

# Инициализация дополнительных опций и настроек
pd.set_option('display.max_columns', 250)
# nltk.download('punkt_tab')
warnings.filterwarnings('ignore')

<hr>
<br>

## Стемминг и Лематизация


- **Стемминг** — процесс нахождения основы для заданного слова, т.е. приведения слова к некоторой базовой форме так, что все различные варианты этого слова могут быть представлены одним и тем же словом (путем удаления окончаний и суффиксов). Например:

    `шапки, шапку, шапок → шапка`

    Стемминг помогает уменьшить сложность текста и улучшить производительность алгоритмов анализа.


- **Лемматизация** — это процесс сопоставления всех различных форм слова с его основой или леммой. Хотя это определение кажется близким к определению стемминга, на самом деле они отличаются. Например:
  
    `позитивные → позитивный`

    В отличие от стемминга, лемматизация сводит слова к их лемме — это более сложный процесс, который учитывает морфологический анализ слов. Лемматизация более точно обрабатывает слова, приводя их к словарной форме.

<hr>

#### Функция для загрузки моделей spaCy

Для обработки текста с помощью библиотеки `spaCy` нужно загрузить натренированную модель. Перечень поддерживаемых языков и наличие моделей можно посмотреть [документации](https://spacy.io/usage/models#languages).

In [2]:
def spacy_model(model_name: str):
    """Функция скачивает указанную языковую модель."""

    try:
        return spacy_load_model(model_name)
    except OSError:
        spacy_download_model(model_name)
        return spacy_load_model(model_name)

Загрузим среднего размера модели для английского и русского языков:
- [en_core_web_md](https://spacy.io/models/en#en_core_web_md) 31Mb
- [ru_core_news_md](https://spacy.io/models/ru#ru_core_news_md) 39Mb

In [3]:
en_spacy_model = spacy_model('en_core_web_md')
ru_spacy_model = spacy_model('ru_core_news_md')

<hr>

#### Кастомный токенизатор для токенизации русских текстов для BoW

Для более качественного векторного представления текста с помощью `Bag of Words` определим свой собственных токенизатор, который будет возвращать леммы слов. Приведение слова к его основной форме (лемме) позволит уменьшить размерность векторов.

Токенизатор будет поддерживать следующий функционал:
- удаление цифр (опция `remove_numbers`, с возможностью отключения) — номера телефонов, года и т.п.;
- удаление знаков пунктуации (опция `remove_punctuation`, с возможностью отключения) — знаки препинания будут являтся шумом в векторной модели;
- удаление прочих символов (опция `remove_symbols`, с возможностью отключения) — смайлики, математические операторы и пр.;
- удаление лишних пробелов (опция `normalize_whitespaces`, с возможностью отключения);
- удаление стоп-слов (опция `keep_stopwords`, с возможностью отключения); стоп-слова — это общеупотребительные слова в языке, которые обычно несут мало смысловой нагрузки (например, "и", "в", "на");
- минимальная длина леммы (`min_length` с возможностью отключения) — удаление лемм с количеством символов меньше заданного порогового значения.

In [4]:
class RussianTokenizer:
    """RussianTokenizer.

    Класс для токенизации русских текстов с помощью
    spaCy. На вход принимается строка, на выходе список
    из лемм.
    """

    def __init__(
        self,
        remove_numbers: bool = True,
        remove_punctuation: bool = True,
        remove_symbols: bool = True,
        normalize_whitespaces: bool = True,
        keep_stopwords: bool = False,
        min_length: int = 1,
    ):
        if remove_numbers:
            self._remove_numbers = re_compile(r'\p{Number}')
        if remove_punctuation:
            self._remove_punctuation = re_compile(r'\p{Punctuation}')
        if remove_symbols:
            self._remove_symbols = re_compile(r'\p{Symbol}')
        if normalize_whitespaces:
            self._normalize_whitespaces = re_compile(r'\s+')

        self._keep_stopwords = keep_stopwords
        self._min_length = min_length
        self._activated_patterns = tuple(
            pattern for attr, pattern in self.__dict__.items() if isinstance(pattern, Pattern)
        )

    def __call__(
        self,
        text_line: str,
        *args: Sequence[Any],
        **kwargs: Mapping[str, Any],
        ) -> Sequence[str]:

        if self._activated_patterns:
            for regex_pattern in self._activated_patterns:
                text_line = regex_pattern.sub(' ', text_line)

        tokenized_doc = ru_spacy_model(text_line)
        if self._keep_stopwords:
            return list(token.lemma_ for token in tokenized_doc if len(token.lemma_) >= self._min_length)
        return list(token.lemma_ for token in tokenized_doc if len(token.lemma_) >= self._min_length and not token.is_stop)

ru_tokenizer = RussianTokenizer(
    min_length=3,
)

#### Функции для загрузки метаинформации по документам и соединения номера документа с доменом и тематикой

In [5]:
def load_meta_data(meta_data_dir: Path) -> Mapping[str, Any]:
    """Функция для загрузки метаинформации по документам."""

    meta_data_path = next(meta_data_dir.glob('meta_data.json'))
    with open(meta_data_path) as file_in:
        meta_data = json.load(file_in)
    return meta_data

######################################################################################

def map_doc_to_subject(meta_data: Mapping[str, Any]) -> Sequence[str]:
    """Функция соединяет номер документа с доменом и тематикой."""

    mapped = []
    for doc, info in meta_data.items():
        doc_topic = info.get('topic', '')
        doc_subject = info.get('subject', '')
        doc_number = int(doc.split('_')[0])
        mapped.append(f'{doc_number}_{doc_topic}_{doc_subject}')
    return mapped

<hr>
<br>

### Стемминг с помощью `NLTK`

Для стемминга токенов на английском языке используется [PorterStemmer](https://www.nltk.org/api/nltk.stem.snowball.html#nltk.stem.snowball.PorterStemmer). По сравнению с другими алгоритмами стемминга он дает наилучший результат и имеет меньший процент ошибок.

In [6]:
remove_punctuation = re_compile(r'\p{Punctuation}')
stemmer = PorterStemmer()
en_text = '"Walking in the woods is pleasant!" - he said happily!'
en_text_no_punct = remove_punctuation.sub('', en_text)
en_tokens = word_tokenize(en_text_no_punct)
en_stemmed_tokens = [stemmer.stem(token) for token in en_tokens]

en_stemming = pd.DataFrame.from_dict(
    data={
        'token': en_tokens,
        'stemmed token': en_stemmed_tokens
    }
)
en_stemming

,token,stemmed token
0,Walking,walk
1,in,in
2,the,the
3,woods,wood
4,is,is
5,pleasant,pleasant
6,he,he
7,said,said
8,happily,happili


**Стемминг на русском языке.**

Для стемминга токенов на других языках используется [Snowball Stemmer](https://www.nltk.org/api/nltk.stem.snowball.html#nltk.stem.snowball.SnowballStemmer). Snowball Stemmer, по сравнению с Porter Stemmer, является мультиязычным. Он поддерживает различные языки и основан на языке программирования Snowball, известном своей эффективностью при обработке небольших строк.

In [7]:
ru_stemmer = SnowballStemmer('russian')
ru_text = 'Съешь еще этих мягких французских булок да выпей чаю.'
ru_text_no_punct = remove_punctuation.sub('', ru_text)
ru_tokens = word_tokenize(ru_text_no_punct)
ru_stemmed_tokens = [ru_stemmer.stem(token) for token in ru_tokens]

ru_stemming = pd.DataFrame.from_dict(
    data={
        'token': ru_tokens,
        'stemmed token': ru_stemmed_tokens
    }
)
ru_stemming

,token,stemmed token
0,Съешь,съеш
1,еще,ещ
2,этих,эт
3,мягких,мягк
4,французских,французск
5,булок,булок
6,да,да
7,выпей,вып
8,чаю,ча


<hr>

### Лемматизация с помощью `spaCy`

<div class="alert alert-info">

Полный перечень атрибутов токена в токенизированном тексте можно посмотреть в [документации](https://spacy.io/api/token#attributes).

In [8]:
# Для английского предложения
en_doc = en_spacy_model('"Walking in the woods is pleasant!" - he said happily!')
en_tokens = [token for token in en_doc if not token.is_punct]

en_lemmatization = pd.DataFrame.from_dict(
    data={
        'token': en_tokens,
        'lemma': [token.lemma_ for token in en_tokens]
    }
)
en_lemmatization

,token,lemma
0,Walking,walk
1,in,in
2,the,the
3,woods,wood
4,is,be
5,pleasant,pleasant
6,he,he
7,said,say
8,happily,happily


<hr>

In [9]:
# Для русского предложения
ru_doc = ru_spacy_model('Съешь еще этих мягких французских булок да выпей чаю.')
ru_tokens = [token for token in ru_doc if not token.is_punct]

ru_lemmatization = pd.DataFrame.from_dict(
    data={
        'token': ru_tokens,
        'lemma': [token.lemma_ for token in ru_tokens]
    }
)
ru_lemmatization

,token,lemma
0,Съешь,съешь
1,еще,ещё
2,этих,этот
3,мягких,мягкий
4,французских,французский
5,булок,булка
6,да,да
7,выпей,выпей
8,чаю,чай


<div class="alert alert-info">

`spaCy` не предоставляет функционал для стемминга, поскольку стемминг неточен. `spaCy` предназначен в основном для использования в продакшене и неточности источник ошибок в продакшн системах. Лемматизация выполняет ту же работу, но более точно с помощью словаря, специфичного для языка, и возвращает точный корень слова.

<hr>
<br>

## One-Hot Encoding

Каждому слову `w` в словаре корпуса присваивается уникальный целочисленный идентификатор  `id`, который находится в диапазоне от 1 до `V`, где V — словарь уникальных слов, полученный из корпуса. Затем каждое слово представляется двоичным вектором V-мерности из нулей и единиц.

In [10]:
sentences = [
    'We need a new truck.',
    'We painted the house green.',
    'We turned on the radio.',
    'Did you play tennis yesterday?'
]

# Создаем перечень уникальных слов (токенов)
text_to_docs = map(lambda line: en_spacy_model(line), sentences)
tokenized_sents = [[token.lemma_ for token in doc if not token.is_punct] for doc in text_to_docs]
unique_tokens = sorted(set(chain.from_iterable(tokenized_sents)))

# Каждому токену в словаре присваиваем индекс и формируем словарь
vocabulary = {token: idx for idx, token in enumerate(unique_tokens, 1)}
print(f'Vocubulary:\n{vocabulary}\n')

# Создаем числовое представления текста для One-Hot энкодера
numerical_data = [[vocabulary[word] for word in doc] for doc in tokenized_sents]

# Векторизация с помощью One-Hot энкодера
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_text = one_hot_encoder.fit_transform(numerical_data)
print('One-Hot Encoded Representation:')
for encoded_line, tokenized_line in zip(encoded_text, tokenized_sents):
    print(f'{encoded_line} | {tokenized_line}')

Vocubulary:
{'a': 1, 'do': 2, 'green': 3, 'house': 4, 'need': 5, 'new': 6, 'on': 7, 'paint': 8, 'play': 9, 'radio': 10, 'tennis': 11, 'the': 12, 'truck': 13, 'turn': 14, 'we': 15, 'yesterday': 16, 'you': 17}

One-Hot Encoded Representation:
[0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0.] | ['we', 'need', 'a', 'new', 'truck']
[0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0.] | ['we', 'paint', 'the', 'house', 'green']
[0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0.] | ['we', 'turn', 'on', 'the', 'radio']
[1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1.] | ['do', 'you', 'play', 'tennis', 'yesterday']


### Недостатки:
- pазмер one-hot вектора прямо пропорционален размеру словаря, и у больших корпусов будут формироваться большие словари. Это приводит к разреженному представлению текста, где большинство записей в векторах являются нулями, что делает его вычислительно неэффективным для хранения, вычисления и обучения (разреженность приводит к переобучению);
- проблема слов не входящих в словарь OOV (out of vocabulary).

<hr>
<br>

## BoW (Bag of Words)

Основная идея заключается в том, чтобы представить рассматриваемый текст в виде мешка (набора) слов, игнорируя порядок и контекст. Основное предположение — текст, принадлежащий к определенному классу характеризуется уникальным набором слов. Если два текста содержат почти одинаковые слова, значит, они принадлежат к одному и тому же классу. Таким образом, анализируя слова, присутствующие в фрагменте текста, можно определить, к какому классу (мешку) он принадлежит.

Реалзиация `BoW` c помощью [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#countvectorizer) из библиотеки `Scikit-learn`

Основные параметры класса `sklearn.feature_extraction.text.CountVectorizer`:
- `input`:
    - `filename` — список файлов которые нужно считать
    - `file` — путь к файлу который нужно считать
    - `content` — последовательность строк или байт
    - **default**=`content`
- `strip_accents` — норамлизация по юникод символам или ascii; **default**=`None`
- `lowercasebool` — приведение к нижнему регистру текста перед токенизацей; **default**=`True`
- `tokenizer` — вызываемый объект; использутся для токенизации текста; **default**=`None`
- `max_features` — словарь будет ограничен количеством токенов, указанным в `max_features`; при этом будут учитываться только наиболее встречающиеся, упорядоченные по частоте в корпусе токены. Если параметр не задан, то словарь строится из всех токенов; **default**=`None`
- `ngram_range` — нижняя и верхняя граница диапазона значений n для различных n-грамм слов или символов, которые будут извлечены; используется для реализации Bag of N-Grams; **default**=(1, 1)

In [11]:
# Проинициализируем векторизатор 
bow_vectorizer = CountVectorizer(
    lowercase=False,
    tokenizer=ru_tokenizer,
    analyzer='word',
    binary=False,
)

In [12]:
corpus = [
    'Яркой визитной карточкой сиамских кошек является их характерный окрас 😋',
    'Мейн-куны одни из самых крупных кошек, их вес может достигать 12 кг!',
    'По характеру сококе подвижные, задорные, любопытные и умные кошки.',
    'Животные средние по своим размерам, вес тела достигает 3-5 кг.',
    'хорошо развитое мускулистое тело среднего размера, вес от 2-х до 4-х килограмм',
]

# Трансформируем строки в мешок слов
bow = bow_vectorizer.fit_transform(corpus)
bow_df = pd.DataFrame(
    data=bow.toarray(),
    columns=bow_vectorizer.get_feature_names_out()
)
bow_df

,вес,визитный,достигать,животное,задорный,карточка,килограмм,кошка,крупный,куна,любопытный,мейн,мускулистый,окрас,подвижный,развитой,размер,сиамский,сококе,средний,тело,умный,характер,характерный,являться,яркий
0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,1
1,1,0,1,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0
3,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0
4,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,1,1,0,0,0,0,0


<br>

Посмотрим на частоты слов из нашего "мешка слов". Отобразим первые 10 слов в порядке убывания по частоте встречаемости в корпусе.

In [13]:
bow_df.sum(axis=0).to_frame(name='frequency').sort_values('frequency', ascending=False).head(10).T

,вес,кошка,достигать,тело,средний,размер,карточка,визитный,задорный,животное
frequency,3,3,2,2,2,2,1,1,1,1


Из полученных частот видно, что слово «вес» встречается в корпусе 3 раза, в строках 2, 4 и 5. А слово «кошка» в строках 1, 2 и 3.

Сформируем «мешок слов» для текстовых файлов, в директории `./data`:

In [14]:
# Сформируем путь к файлам и отсортируем список файлов по имени
data_dir = Path('data').resolve()
doc_files = sorted(data_dir.glob('*_doc*'), key=lambda file: int(file.name.split('_')[0]))

# Проинициализируем векторизатор с аргументом `filename` для считывания файлов 
docs_bow_vectorizer = CountVectorizer(
    input='filename',
    strip_accents='unicode',
    lowercase=True,
    tokenizer=ru_tokenizer,
    analyzer='word',
    max_features=350,
    binary=False,
)
# Трансформируем документы в мешок слов
docs_bow = docs_bow_vectorizer.fit_transform(doc_files)
docs_bow_df = pd.DataFrame(
    data=docs_bow.toarray(),
    columns=docs_bow_vectorizer.get_feature_names_out()
)
docs_bow_df

,atlas,betula,catus,familiaris,fel,felis,marvel,pro,silvestris,абиссинский,авто,автомат,автомобиль,автор,азия,аллерген,аллергик,аллергия,америка,американский,анализ,англ,археологический,архитектор,башня,беларусь,белок,белорусский,белый,берег,береза,берёза,берёзовый,бесхвостый,биолог,ближний,больший,большинство,большой,бородавчатый,важный,вариант,василёк,век,версия,вес,взрослый,вид,включать,владелец,вместе,внимание,вода,водоём,возраст,волк,восток,время,встречаться,выпуск,высокий,высоко,высота,высотои,генетический,говорить,год,головастик,город,гриб,громовержец,грудной,группа,грызун,давать,два,двигатель,девушка,деиствительно,делать,деревня,дерево,диапазон,дикий,дикои,дилер,длина,длинный,днк,доллар,дом,домашнеи,домашний,достаточно,достигать,достигнуть,древесный,древний,европа,животное,жидкость,жизнь,жить,зависеть,задний,здание,земля,зеркало,значение,зона,зрение,зритель,игра,известный,иллиноис,иметь,исполнение,использовать,использоваться,исследование,источник,капитан,киновселеннои,книга,кожа,...,позвонок,позвоночник,поздний,показать,покров,пол,полагать,полностью,получить,популярный,порода,последний,посмотреть,посёлок,похожий,пояс,правда,правило,практически,предок,предполагать,представителеи,представитель,представлять,привод,примерно,природа,причина,пробег,проблема,проект,произойти,происходить,происхождение,пространство,проходить,пушистый,пять,работать,раз,развитие,различный,размер,разный,раит,раита,распространить,ребро,результат,река,речь,род,роль,русский,рынок,своеи,сделать,семеиства,сибирский,симптом,скелет,слово,случай,собака,собои,современный,согласно,содержание,соединить,составлять,состояние,состоять,способ,способный,сравнение,среда,становиться,старый,стиль,стоить,стоянка,страж,страна,строение,существовать,считать,считаться,счёт,сша,сюжет,такои,тело,термин,территория,точка,три,тыс,тысяча,увеличить,указывать,условие,установить,учёный,факт,фильм,форма,фрэнк,характер,хвост,хищник,хороший,цена,часть,человек,человеческий,челюсть,череп,число,чёрный,шерсть,экземпляр,эмоциональный,этои,являться,язык
0,0,0,7,0,0,19,0,0,9,0,0,0,0,0,0,0,0,0,0,0,1,1,2,0,0,0,0,0,0,0,0,0,0,0,0,4,2,2,0,0,0,0,0,1,0,5,1,5,0,0,2,0,0,1,0,3,4,7,3,0,2,1,0,0,3,1,14,0,0,0,0,4,7,6,1,4,0,0,1,0,0,0,1,9,4,0,4,1,3,0,0,3,18,1,4,0,0,2,4,11,6,5,1,0,1,0,0,0,0,1,2,0,0,1,0,2,0,3,1,2,3,0,0,1,0,...,4,3,1,2,2,1,1,1,1,2,3,0,0,0,0,0,0,1,1,5,2,0,1,0,0,2,1,0,0,0,0,5,2,1,0,0,0,2,0,2,2,2,1,2,0,0,1,3,1,0,0,1,0,4,0,0,0,1,1,0,4,7,2,5,2,1,2,0,2,4,0,5,0,5,1,1,2,0,0,0,0,0,0,0,0,4,1,0,0,0,0,5,0,2,2,2,2,4,0,0,2,3,7,3,0,0,0,0,2,5,0,0,1,16,2,3,5,2,0,1,1,2,0,11,10
1,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,2,0,0,0,1,1,0,1,0,0,1,1,0,0,1,2,0,0,0,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,2,0,0,0,0,0,7,0,3,0,1,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,9,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,2,0,0,0,3,2,1,0,0,0,0,0,1,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,3,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,2,0,0,0,0,0,2,0,0,0,0,0,3,0,0,3,0,0
2,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,7,4,12,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,2,0,0,2,0,0,0,2,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3,0,4,0,0,1,0,0,0,6,1,2,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,...,0,0,0,0,1,1,0,3,0,0,5,0,0,0,0,0,1,1,0,0,0,3,1,0,0,0,1,2,0,0,0,0,0,0,0,0,4,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1,8,6,0,0,0,1,0,0,1,3,0,0,2,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,7,0,0,0,0,0
3,0,0,0,4,0,0,0,0,0,0,0,0,0,0,5,0,0,0,3,2,5,7,3,0,0,0,0,0,0,0,0,0,0,0,4,2,0,0,1,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,5,19,3,5,2,0,0,1,0,1,5,0,23,0,0,0,0,0,4,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,4,0,0,3,8,0,0,0,0,5,2,6,0,1,1,0,0,0,0,0,1,0,3,0,1,1,0,0,0,0,2,3,3,0,0,0,0,...,0,0,0,5,0,2,3,0,0,1,5,1,0,0,1,0,0,0,0,9,4,0,2,1,0,0,0,2,0,0,0,6,4,4,0,0,0,1,0,0,0,1,2,6,0,0,1,0,4,2,2,0,1,0,0,0,2,1,0,0,0,5,0,62,1,4,1,0,0,0,0,0,0,0,0,1,0,2,0,0,4,0,0,1,3,3,1,0,0,0,0,0,1,1,

Отобразим первые 15 слов в порядке убывания по частоте встречаемости в текстовых файлах.

In [15]:
docs_bow_df.sum(axis=0).to_frame(name='frequency').sort_values('frequency', ascending=False).head(15).T

,кошка,год,собака,лягушка,человек,вид,животное,домашний,озеро,берёза,время,название,порода,волк,являться
frequency,126,73,68,51,40,39,36,31,30,30,25,25,22,22,22


Отобразим последние 15 слов в текстовых файлах.

In [16]:
docs_bow_df.sum(axis=0).to_frame(name='frequency').sort_values('frequency', ascending=True).head(15).T

,эмоциональный,betula,familiaris,чёрный,белый,белок,биолог,берег,термин,сша,сравнение,способ,соединить,указывать,увеличить
frequency,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4


#### Сравним анализируемые документы на похожесть с помощью косинусного расстояния

In [17]:
# Произведем сжатие разреженных векторов
sparse_matrix = sparse.csr_matrix(docs_bow.toarray())
# Рассчитаем схожесть между векторами
cosine_similarities = cosine_similarity(sparse_matrix)

meta_data = load_meta_data(data_dir)
doc_subject_names = map_doc_to_subject(meta_data)

# Сформируем датафрейм с коэффициентами похожести
docs_similarity_df = pd.DataFrame(
    cosine_similarities,
    columns=doc_subject_names,
    index=doc_subject_names,
)

In [18]:
matrix_similarity = docs_similarity_df.where(
    np.tril(np.ones(docs_similarity_df.shape), k=-1).astype(np.bool)
)
styled_similarity = (matrix_similarity
    .style
    .background_gradient(cmap='YlGnBu')
    .highlight_null('white')
    .format("{:.2%}", na_rep="")
)
styled_similarity

,1_animals_cat,2_animals_cat,3_animals_cat,4_animals_dog,5_animals_frog,6_movie_thunderbolts*,7_real_estate_skyscraper,8_nature_lake_naroch,9_auto_geely,10_plants_birch
1_animals_cat,,,,,,,,,,
2_animals_cat,71.79%,,,,,,,,,
3_animals_cat,43.15%,48.37%,,,,,,,,
4_animals_dog,21.25%,14.61%,9.18%,,,,,,,
5_animals_frog,10.01%,9.19%,8.89%,5.00%,,,,,,
6_movie_thunderbolts*,10.82%,7.80%,4.19%,13.33%,5.35%,,,,,
7_real_estate_skyscraper,9.00%,11.19%,6.60%,12.36%,5.35%,14.62%,,,,
8_nature_lake_naroch,5.58%,6.94%,3.43%,7.12%,4.95%,7.92%,8.57%,,,
9_auto_geely,7.40%,9.65%,3.14%,13.77%,3.01%,15.22%,11.98%,7.78%,,
10_plants_birch,3.59%,6.58%,4.04%,2.80%,8.24%,3.45%,5.78%,4.97%,1.41%,


<hr>
<br>

<div class="alert alert-danger">

А что если мы не будем заморачиваться с реализаций собственного токенизатора и откажемся также от нормализации по юникоду?

In [19]:
default_docs_bow_vectorizer = CountVectorizer(
    input='filename',
    lowercase=True,
    analyzer='word',
    binary=False,
)
default_docs_bow = default_docs_bow_vectorizer.fit_transform(doc_files)
default_docs_bow_df = pd.DataFrame(
    data=default_docs_bow.toarray(),
    columns=default_docs_bow_vectorizer.get_feature_names_out()
)

default_sparse_matrix = sparse.csr_matrix(default_docs_bow.toarray())
default_cosine_similarities = cosine_similarity(default_sparse_matrix)

default_docs_similarity_df = pd.DataFrame(
    default_cosine_similarities,
    columns=doc_subject_names,
    index=doc_subject_names,
)

default_matrix_similarity = default_docs_similarity_df.where(
    np.tril(np.ones(default_docs_similarity_df.shape), k=-1).astype(np.bool)
)
default_styled_similarity = (default_matrix_similarity
    .style
    .background_gradient(cmap='YlGnBu')
    .highlight_null('white')
    .format("{:.2%}", na_rep="")
)
default_styled_similarity

,1_animals_cat,2_animals_cat,3_animals_cat,4_animals_dog,5_animals_frog,6_movie_thunderbolts*,7_real_estate_skyscraper,8_nature_lake_naroch,9_auto_geely,10_plants_birch
1_animals_cat,,,,,,,,,,
2_animals_cat,51.19%,,,,,,,,,
3_animals_cat,34.99%,35.67%,,,,,,,,
4_animals_dog,41.66%,26.05%,28.06%,,,,,,,
5_animals_frog,30.44%,25.78%,30.03%,26.94%,,,,,,
6_movie_thunderbolts*,35.92%,29.72%,41.28%,36.90%,36.68%,,,,,
7_real_estate_skyscraper,32.30%,29.18%,40.32%,37.10%,28.70%,50.62%,,,,
8_nature_lake_naroch,26.26%,26.71%,30.54%,27.37%,27.73%,40.09%,38.55%,,,
9_auto_geely,23.13%,20.93%,28.64%,26.05%,23.58%,39.03%,33.64%,29.63%,,
10_plants_birch,24.39%,23.92%,25.73%,23.24%,26.17%,37.61%,33.06%,31.21%,25.76%,


<div class="alert alert-info">

По коэффициентам похожести видно, что при дефолтных параметрах, документы которые относятся к разным домена стали более похожи между собой. Из этого следует, что дополнительная обработка и подбор параметров является важной составляющей.

<div class="alert alert-warning">

<strong>Нужна ли лемматизация?</strong> Посмотрим на следующих примерах.

In [20]:
# Bag of Words с лемматизацей
lemmas_docs_bow_vectorizer = CountVectorizer(
    input='filename',
    lowercase=True,
    tokenizer=ru_tokenizer,
    analyzer='word',
)

lemma_docs_bow = lemmas_docs_bow_vectorizer.fit_transform(doc_files)
lemma_vocabulary = sorted(lemmas_docs_bow_vectorizer.get_feature_names_out())
lemma_vocabulary[767:777]

['животное',
 'жидкость',
 'жизнедеятельность',
 'жизненный',
 'жизнь',
 'жильё',
 'житель',
 'жить',
 'журналист',
 'жёлтый']

In [21]:
# Bag of Words без лемматизации
raw_docs_bow_vectorizer = CountVectorizer(
    input='filename',
    lowercase=True,
    analyzer='word',
)

raw_docs_bow = raw_docs_bow_vectorizer.fit_transform(doc_files)
raw_vocabulary = sorted(raw_docs_bow_vectorizer.get_feature_names_out())
raw_vocabulary[1304:1314]

['животного',
 'животное',
 'животные',
 'животным',
 'животных',
 'живут',
 'живущих',
 'живших',
 'жидкостей',
 'жидкости']

<div class="alert alert-info">


Из среза словарей видно, что в `BoW` без лемматизации в словаре несколько форм слова `животное` → ['животное', 'животные', животным', 'животных'], а в словаре `BoW` с лемматизацией только одно слово `животное`. Таким образом, лемматизация позволяет наполнить словарь более разнообразными словами и уменьшить размер самого словаря что является важным для огромных текстовых корпусов.

<hr>
<br>

## BoN (Bag of N-Grams)
